In [ ]:
##Arbitrary Style Transfer Code

Importing all the modules

In [1]:
import numpy as np
from os import listdir, mkdir, sep
from os.path import join, exists, splitext
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [2]:
!wget http://images.cocodataset.org/zips/test2014.zip
!unzip *.zip
!rm *.zip
!mkdir -p test2014/1
!mv test2014/*.jpg test2014/1/

Streaming output truncated to the last 5000 lines.
 extracting: test2014/COCO_test2014_000000112376.jpg  
 extracting: test2014/COCO_test2014_000000484597.jpg  
 extracting: test2014/COCO_test2014_000000135715.jpg  
 extracting: test2014/COCO_test2014_000000456549.jpg  
 extracting: test2014/COCO_test2014_000000188376.jpg  
 extracting: test2014/COCO_test2014_000000169328.jpg  
 extracting: test2014/COCO_test2014_000000015971.jpg  
 extracting: test2014/COCO_test2014_000000294000.jpg  
 extracting: test2014/COCO_test2014_000000481459.jpg  
 extracting: test2014/COCO_test2014_000000230069.jpg  
 extracting: test2014/COCO_test2014_000000394792.jpg  
 extracting: test2014/COCO_test2014_000000246684.jpg  
 extracting: test2014/COCO_test2014_000000138471.jpg  
 extracting: test2014/COCO_test2014_000000065390.jpg  
 extracting: test2014/COCO_test2014_000000434419.jpg  
 extracting: test2014/COCO_test2014_000000279982.jpg  
 extracting: test2014/COCO_test2014_000000270728.jpg  
 extracting: t

In [3]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [4]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


# New Section

In [5]:
enc=nn.Sequential(*list(model.features.children())[:21])
for i in enc.parameters():
  i.requires_grad = False
enc

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [6]:
# Use Sequential to define decoder [Just reverse of vgg with pooling replaced by nearest neigbour upscaling]
dec = nn.Sequential(
nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect' ),
nn.ReLU(inplace=True),
nn.Upsample(scale_factor=2,mode='nearest'),
nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Upsample(scale_factor=2,mode='nearest'),
nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Upsample(scale_factor=2,mode='nearest'),
nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True),
nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode='reflect'),
nn.ReLU(inplace=True)
)

In [7]:
dec

Sequential(
  (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (1): ReLU(inplace=True)
  (2): Upsample(scale_factor=2.0, mode=nearest)
  (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (4): ReLU(inplace=True)
  (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (6): ReLU(inplace=True)
  (7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (8): ReLU(inplace=True)
  (9): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (10): ReLU(inplace=True)
  (11): Upsample(scale_factor=2.0, mode=nearest)
  (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (13): ReLU(inplace=True)
  (14): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (15): ReLU(inplace=True)
  (16): Upsample(scale_fac

In [8]:
# pipeline

In [9]:
class Encoder_Decoder(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    
    self.style_features = []
    self.style_layers = [1, 6, 11, 20] # relu1_1, relu2_1, relu3_1, relu4_1
    for i in self.style_layers:
      self.encoder._modules[str(i)].register_forward_hook(self.style_feature_hook)

  def style_feature_hook(self, module, input, output):
    self.style_features.append(output)

  def forward(self, image):

    self.content_in = self.encoder(image)
    self.style_features = []

    return self.decoder(self.content_in)

In [10]:
def read_image(path, size=None, gray=False):
    img = cv2.imread(path)
    if gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if size != None:
        img = cv2.resize(img, size)
    return img

In [33]:
scale = lambda x : (255 * (x - x.min())) / (x.max() - x.min())

def save_tensor_image(img, name, de_normalize=True):

  if de_normalize:
    img = torch.round(torch.dstack((scale(img[0]), scale(img[1]), scale(img[2])))).type(torch.uint8)
    img = Image.fromarray(img.cpu().detach().numpy())
    img.save(name)
  else:
    save_image(img, name)

In [12]:
from PIL import Image
# lion = Image.open('drive/MyDrive/images/style/1/lion.jpg')
# lion = Image.open('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fs-media-cache-ak0.pinimg.com%2Foriginals%2Fad%2F21%2F86%2Fad2186c3301997a31780d5ab600d71c9.jpg&f=1&nofb=1')

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: x.to(device))
])

In [14]:
device

device(type='cuda', index=0)

In [15]:
enc = enc.to(device)
dec = dec.to(device)

In [ ]:
# train = DataLoader(ImageFolder('drive/MyDrive/images/content', transform=preprocess), batch_size=1, shuffle=True, num_workers=0)
# test = DataLoader(ImageFolder('drive/MyDrive/images/style', transform=preprocess), batch_size=1, shuffle=True, num_workers=0)

In [ ]:
# x = 0
# for i in train:
#   save_image(i[0], f"{x}.jpg")
#   plt.imshow(Image.open(f"{x}.jpg"))
#   plt.show()

In [ ]:
# x = 0
# for i in test:
#   save_image(i[0], f"{x}.jpg")
#   plt.imshow(Image.open(f"{x}.jpg"))
#   plt.show()

In [16]:
def trainModel(Model, Loss, trainImagesPath, testImagesPath, numImages = 100, batchSize=1, epochs=5, learningRate=0.001, weightDecay=1e-3):

  train_images = DataLoader(ImageFolder(trainImagesPath, transform=preprocess), batch_size=batchSize, shuffle=True, num_workers=0)
  # test_images = DataLoader(ImageFolder(testImagesPath, transform=preprocess), batch_size=batchSize, shuffle=True, num_workers=0)

  model = Model(enc, dec).to(device)
  loss_fn = Loss().to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)

  for e in range(epochs):
    epoch_loss = 0.0
    i=0.0
    for img in train_images:
      img = img[0]
      i+=1
      if(i > numImages): 
        break
      print("\r", i, end="")
      optimizer.zero_grad()

      decoded = model(img)
    
      content_in = model.encoder(img)
      content_out = model.encoder(decoded)
      styles_in = model.style_features[:4]
      styles_out = model.style_features[4:]

      loss = loss_fn(content_in, content_out, styles_in, styles_out)
      loss.backward()
      
      optimizer.step()
      epoch_loss += loss.item()
      del content_in, content_out, styles_in, styles_out, decoded, loss
      # save_image(decoded, f"tmp_{i}.jpg")
    print("\nEpoch", e, end=": ")
    print("Epoch Loss = ", epoch_loss/len(train_images))

  
  torch.save(model.state_dict(), 'enc_dec_model')
  return model



In [17]:
class ContentStyleLoss(nn.Module):
  def __init__(self, lam=0.5):
    super().__init__()
    self.lam = lam

  def forward (self, content_in, content_out, styles_in, styles_out):
    contentLoss = torch.norm(content_out - content_in)
    styleLoss = np.sum([
                           torch.linalg.norm(torch.mean(styles_out[i], (2, 3)) - torch.mean(styles_in[i], (2,3))) + 
                           torch.linalg.norm(torch.std(styles_out[i], axis=(2, 3), unbiased=False) - torch.std(styles_in[i], axis=(2, 3), unbiased=False)) 
                           for i in range(len(styles_in))
    ])

    return contentLoss + self.lam*styleLoss

In [18]:
md = trainModel(Encoder_Decoder, ContentStyleLoss, 'test2014', 'drive/MyDrive/images/style', numImages=200, epochs=100)

 200.0
Epoch 0: Epoch Loss =  13.231232908659374
 200.0
Epoch 1: Epoch Loss =  12.75410630460607
 200.0
Epoch 2: Epoch Loss =  11.884526564775253
 200.0
Epoch 3: Epoch Loss =  11.718738837788838
 200.0
Epoch 4: Epoch Loss =  11.39862987562749
 200.0
Epoch 5: Epoch Loss =  11.135499045291903
 200.0
Epoch 6: Epoch Loss =  10.995887384560852
 200.0
Epoch 7: Epoch Loss =  10.91749464567127
 200.0
Epoch 8: Epoch Loss =  10.824213489974516
 200.0
Epoch 9: Epoch Loss =  11.043475170404468
 200.0
Epoch 10: Epoch Loss =  10.557302720603733
 200.0
Epoch 11: Epoch Loss =  10.467007536475897
 200.0
Epoch 12: Epoch Loss =  10.40487092580041
 200.0
Epoch 13: Epoch Loss =  10.74844928042133
 200.0
Epoch 14: Epoch Loss =  10.402143406496686
 200.0
Epoch 15: Epoch Loss =  10.50687277713778
 200.0
Epoch 16: Epoch Loss =  10.24212774617039
 200.0
Epoch 17: Epoch Loss =  10.269639037112967
 200.0
Epoch 18: Epoch Loss =  9.944060829485363
 200.0
Epoch 19: Epoch Loss =  10.144161808802115
 200.0
Epoch 20: E

In [39]:
preprocess_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.to(device)),
    transforms.Resize(256),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # transforms.Lambda(lambda x: x.unsqueeze(0))
])

model = Encoder_Decoder(enc, dec).to(device)
model.load_state_dict(torch.load('enc_dec_model'))
loss = ContentStyleLoss()

In [ ]:
# lion = Image.open('drive/MyDrive/images/style/1/lion.jpg')
# model_in = preprocess_test(lion)

# op = model(model_in)
# save_image(model_in, 'test_in.jpg')
# save_image(op, 'test_out.jpg')

# content_in = model.encoder(model_in)
# content_out = model.encoder(op)
# styles_in = model.style_features[:4]
# styles_out = model.style_features[4:]
# l = loss(content_in, content_out, styles_in, styles_out)
# print(l.item())

In [36]:
model.eval()

Encoder_Decoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(

In [40]:
test_images = DataLoader(ImageFolder('drive/MyDrive/images/content', transform=preprocess_test), batch_size=1, shuffle=True, num_workers=0)
x = 1
for i in test_images:
  i = i[0]
  # print(i.shape)
  decoded = model(i)
  content_in = model.encoder(i)
  content_out = model.encoder(decoded)
  styles_in = model.style_features[:4]
  styles_out = model.style_features[4:]

  l = loss(content_in, content_out, styles_in, styles_out)
  print(l.item())
  i = i[0]
  decoded = decoded[0]
  save_tensor_image(i, f'in_{x}.jpg')
  save_tensor_image(decoded, f'out{x}.jpg')
  x += 1
  del content_in, content_out, styles_in, styles_out


3040.280517578125
3207.093505859375
2144.852294921875
2397.80712890625
1675.1787109375
